# GPT in Generating Shallow Feedback for Alien Relations Processes
 <!-- title: Your social media posts VS Controversial CEOs' - with NLP -->
Generative Pre-trained Transformer 3 is an autoregressive language model that uses deep learning to produce human-like text. The architecture is a standard Transformer network with the unprecedented size of 2048-token-long context and 175 billion parameters
<!-- @import "[TOC]" {cmd="toc" depthFrom=1 depthTo=6 orderedList=false} -->

In [4]:
# Pip below for Kaggle and online notebooks.
# !pip install ipywidgets
# !pip install sklearn
# !pip install spacy
import numpy as np
import pandas as pd

%matplotlib inline

RANDOM_STATE = 42

# Remove anywarning texts from notebooks.
import warnings
warnings.filterwarnings("ignore")

# data = pd.read_csv('/kaggle/input/communications/communications.csv')
data = pd.read_csv('./feedbacks.csv')
data = data.dropna()
data = data.sample(10, random_state=RANDOM_STATE)


In [5]:
import sklearn
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer
from transformers import TextDataset, DataCollatorForLanguageModeling

train_path = 'train.txt'
test_path = 'test.txt'
train, test = train_test_split(data['feedback'],test_size=0.15) 
with open(train_path, 'w') as filehandle:
    for listitem in train:
        filehandle.write('%s\n' % listitem)
with open(test_path, 'w') as filehandle:
    for listitem in test:
        filehandle.write('%s\n' % listitem)

tokenizer = AutoTokenizer.from_pretrained("gpt2")

def load_dataset(train_path,test_path,tokenizer):
    train_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=train_path,
          block_size=20)
     
    test_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=test_path,
          block_size=20)   
    
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )
    return train_dataset,test_dataset,data_collator


train_dataset, test_dataset, data_collator = load_dataset(
    train_path, test_path, tokenizer)


Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file https://huggingface.co/gpt2/resolve/main/config.json from cache at C:\Users\bigworker/.cache\huggingface\transformers\fc674cd6907b4c9e933cb42d67662436b89fa9540a1f40d7c919d0109289ad01.7d2e0efa5ca20cef4fb199382111e9d3ad96fd77b849e1d4bed13a66e1336f51
Model config GPT2Config {
  "_name_or_path": "gpt2",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj

In [6]:
from transformers import Trainer, TrainingArguments, AutoModelWithLMHead

model = AutoModelWithLMHead.from_pretrained("gpt2")

training_args = TrainingArguments(
    output_dir="/content",  
    overwrite_output_dir=True,  
    num_train_epochs=3,  
    per_device_train_batch_size=1, 
    per_device_eval_batch_size=1, 
    eval_steps=400,
    save_steps=800,  
    warmup_steps=500
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

trainer.train()
trainer.save_model()

loading configuration file https://huggingface.co/gpt2/resolve/main/config.json from cache at C:\Users\bigworker/.cache\huggingface\transformers\fc674cd6907b4c9e933cb42d67662436b89fa9540a1f40d7c919d0109289ad01.7d2e0efa5ca20cef4fb199382111e9d3ad96fd77b849e1d4bed13a66e1336f51
Model config GPT2Config {
  "_name_or_path": "gpt2",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specifi

  0%|          | 0/1824 [00:00<?, ?it/s]

{'loss': 3.9616, 'learning_rate': 5e-05, 'epoch': 0.82}


Saving model checkpoint to /content\checkpoint-800
Configuration saved in /content\checkpoint-800\config.json
Model weights saved in /content\checkpoint-800\pytorch_model.bin


{'loss': 3.2384, 'learning_rate': 3.11178247734139e-05, 'epoch': 1.64}
{'loss': 2.6673, 'learning_rate': 1.2235649546827795e-05, 'epoch': 2.47}


Saving model checkpoint to /content\checkpoint-1600
Configuration saved in /content\checkpoint-1600\config.json
Model weights saved in /content\checkpoint-1600\pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)


Saving model checkpoint to /content
Configuration saved in /content\config.json


{'train_runtime': 3005.506, 'train_samples_per_second': 0.607, 'train_steps_per_second': 0.607, 'train_loss': 3.145091274328399, 'epoch': 3.0}


Model weights saved in /content\pytorch_model.bin


In [8]:
from transformers import pipeline

generator = pipeline('text-generation', model='/content', tokenizer='gpt2')
generator('really this guy')

loading configuration file /content\config.json
Model config GPT2Config {
  "_name_or_path": "/content",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "torch_dtype": "float32",
  "transformers_version": "4.20.1",
  "use_cache": true,
  "vocab_size": 50257
}

loading configur

[{'generated_text': 'really this guy is constantly under-performing and needs to be nurtured to a higher standard.\nIvanna Gray has shown a aptitude for achieving high standards with this work. Although she has delivered relatively little in terms of results, she is capable'}]

# Concluding our Analysis


## References

- https://huggingface.co/gpt2-large
- https://huggingface.co/docs/transformers/training 

## Github and Kaggle

Article here is also available on [Github](https://github.com/adamd1985/articles/blob/main/nlp_bs_meter/nlp_intro.ipynb) and [Kaggle](https://www.kaggle.com/code/addarm/authorship-identification-you-vs-ceos)